In [3]:
import pyspark
from operator import add, sub
from time import sleep
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.streaming.kafka import KafkaUtils
import json
from pyspark import SparkContext, SparkConf


In [6]:
# conf = SparkConf().set("spark.jars", "/path-to-jar/spark-streaming-kafka-0-8-assembly_2.11-2.2.1.jar")

# sc = SparkContext( conf=conf)
# spark = SparkSession.builder.appName('my_awes').master("spark://bphani.local:7077").config("spark.jars", "/Users/phaneendra/Downloads/spark-sql-kafka-0-10_2.11-2.4.4.jar,/Users/phaneendra/Downloads/kafka-clients-2.4.0.jar").getOrCreate()
spark = SparkSession.builder.appName('my_awes').master("local").config("spark.jars", "/Users/phaneendra/Downloads/spark-sql-kafka-0-10_2.11-2.4.4.jar,/Users/phaneendra/Downloads/kafka-clients-2.4.0.jar").getOrCreate()

spark = SparkSession.builder.appName('my_awes').master("yarn").config("spark.yarn.keytab","").config("spark.yarn.principal","").config("spark.yarn.access.hadoopFileSystems","webhdfs://172.16.109.117:9870").getOrCreate()


from pipelineblocksdk.util.kerbUtil import get_authentication_details, generate_ticket_granting_ticket
auth_token = "6d72d2cf-45b6-464c-9811-d6cf21574ece"
auth_name = "hive_new"
keytab_file_path, kerb_conf_path, user_realm = get_authentication_details(auth_token, auth_name)
generate_ticket_granting_ticket({"userAuthToken":auth_token}, auth_name)

spark = SparkSession.builder.appName('my_awes').master("yarn").config("spark.yarn.keytab",keytab_file_path).config("spark.yarn.principal",user_realm).config("spark.yarn.access.hadoopFileSystems","webhdfs://172.16.109.117:9870").getOrCreate()


path = "webhdfs://192.168.48.217:50070/testing123.csv"

In [3]:
#kerbaros
from pipelineblocksdk.util.kerbUtil import get_authentication_details, generate_ticket_granting_ticket
auth_token = "6d72d2cf-45b6-464c-9811-d6cf21574ece"
auth_name = "hive_new"
keytab_file_path, kerb_conf_path, user_realm = get_authentication_details(auth_token, auth_name)
generate_ticket_granting_ticket({"userAuthToken":auth_token}, auth_name)


#HDFS client
import requests
from hdfs import InsecureClient
from hdfs.ext.kerberos import KerberosClient 
session = requests.Session()
principal = 'phani'
full_host = f"http://192.168.48.217:50070/"
client = KerberosClient(url=full_host, session=session, mutual_auth='OPTIONAL', principal=principal)
client.list('/')

#Additional operations
client.download('/user/phani/100000SalesRecords1.csv', '/opt/temp.csv')

client.upload('/user/cloudera/100000SalesRecords1.csv', '/bigbrain/data/100000SalesRecords1.csv', n_threads=4, chunk_size=5000000, cleanup=True, overwrite=True)

In [4]:
# principal = generate_ticket_granting_ticket(block_params, connection_params["authName"])
import subprocess
import time
import requests

retries = 10
keytab_file_path = '/Users/phaneendra/Downloads/phani.keytab'
kerb_conf_path = '/Users/phaneendra/Downloads/krb5.conf'
user_realm = 'phani@CLOUDERA'
subprocess.Popen(["cp", kerb_conf_path, "/etc/"]).wait(timeout=10)
subprocess.Popen(["kdestroy", "-a"]).wait(timeout=10)
time.sleep(2)
subprocess.Popen(["kinit", "-kt", keytab_file_path, user_realm]).wait(timeout=10)
is_kinit_done = (subprocess.Popen(["klist"]).wait(timeout=10) == 0)

while (not is_kinit_done) and (retries > 0):
    subprocess.Popen(["kinit", "-kt", keytab_file_path, user_realm]).wait(timeout=10)
    is_kinit_done = (subprocess.Popen(["klist"]).wait(timeout=10) == 0)
    retries -= 1


print(is_kinit_done)

False


HdfsError: Authentication failure. Check your credentials.

In [ ]:
from pipelineblocksdk.util.kerbUtil import get_authentication_details
auth_token = "6d72d2cf-45b6-464c-9811-d6cf21574ece"
auth_name = "hive_new"
keytab_file_path, kerb_conf_path, user_realm = get_authentication_details(auth_token, auth_name)

In [1]:

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('my_awes').master("local").config("spark.jars", "/Users/phaneendra/Downloads/spark-sql-kafka-0-10_2.11-2.4.4.jar,/Users/phaneendra/Downloads/kafka-clients-2.4.0.jar,/Users/phaneendra/Downloads/spark-avro_2.11-2.4.5.jar,/Users/phaneendra/Downloads/avro-1.9.2.jar").getOrCreate()

agg_match_stats = '/Users/phaneendra/Downloads/100000SalesRecords1.csv'
adf = spark.read.csv(agg_match_stats, inferSchema=True, header=True)
adf.columns

['Region',
 'Country',
 'Item Type',
 'Sales Channel',
 'Order Priority',
 'Order Date',
 'Order ID',
 'Ship Date',
 'Units Sold',
 'Unit Price',
 'Unit Cost',
 'Total Revenue',
 'Total Cost',
 'Total Profit']

In [ ]:
# csv - > bzip2, deflate, uncompressed, lz4, gzip, snappy, none.
# parquet - > uncompressed, gzip, lzo, snappy, none
# orc -> uncompressed, lzo, snappy, zlib, none.

In [2]:
file_format="avro"
codec="none"
col_list = [x.replace(' ','_').upper() for x in adf.columns]
adf = adf.toDF(*col_list)
adf.write.mode("overwrite").option("compression",codec).format(file_format).save(f"output_data.{file_format}")


Py4JJavaError: An error occurred while calling o51.save.
: java.lang.NoSuchMethodError: org.apache.avro.Schema.createUnion([Lorg/apache/avro/Schema;)Lorg/apache/avro/Schema;
	at org.apache.spark.sql.avro.SchemaConverters$.toAvroType(SchemaConverters.scala:185)
	at org.apache.spark.sql.avro.SchemaConverters$$anonfun$5.apply(SchemaConverters.scala:176)
	at org.apache.spark.sql.avro.SchemaConverters$$anonfun$5.apply(SchemaConverters.scala:174)
	at scala.collection.Iterator$class.foreach(Iterator.scala:893)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1336)
	at scala.collection.IterableLike$class.foreach(IterableLike.scala:72)
	at org.apache.spark.sql.types.StructType.foreach(StructType.scala:99)
	at org.apache.spark.sql.avro.SchemaConverters$.toAvroType(SchemaConverters.scala:174)
	at org.apache.spark.sql.avro.AvroFileFormat$$anonfun$7.apply(AvroFileFormat.scala:118)
	at org.apache.spark.sql.avro.AvroFileFormat$$anonfun$7.apply(AvroFileFormat.scala:118)
	at scala.Option.ge

In [33]:
def customFunction(rows):
    with open('/Users/phaneendra/IdeaProjects/notes/nexon/some_out.txt', 'w') as fip:
        for row in rows:
            my_list = list(row.asDict().values())
            csv_str = ",".join(str(item) for item in my_list)
            fip.write(csv_str+'\n')

adf.rdd.foreachPartition(customFunction)

In [ ]:
%%time
adf.selectExpr("to_json(struct(*)) AS value") \
  .write \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092") \
  .option("topic", "test_kafka_spark_3") \
  .option("checkpointLocation", "/Users/phaneendra/IdeaProjects/notes/nexon/checkpoint.txt") \
  .save()

print('Done')

In [ ]:
%%time
adf.count()

In [ ]:
%%time
accum = spark.sparkContext.accumulator(0)
def do_count(x):
    accum.add(1)
    return x
adf_count = adf.rdd.map(do_count).toDF()

adf_count.selectExpr("to_json(struct(*)) AS value") \
  .write \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092") \
  .option("topic", "test_kafka_spark_3") \
  .option("checkpointLocation", "/Users/phaneendra/IdeaProjects/notes/nexon/checkpoint.txt") \
  .save()

print('Done')

In [ ]:
accum

In [ ]:
agg_match_stats = '/Users/phaneendra/Downloads/agg_match_stats_0.csv'
kill_match_stats = '/Users/phaneendra/Downloads/kill_match_stats_final_0.csv'

In [ ]:
adf = spark.read.csv(agg_match_stats, inferSchema=True, header=True)
kdf = spark.read.csv(kill_match_stats, inferSchema=True, header=True)
print(adf.schema)
print(kdf.schema)

In [ ]:
print(adf.show())

In [ ]:
x.columns

In [ ]:
adf.columns

In [ ]:
print(adf.columns)
print(kdf.columns)
print(str(adf.schema.json()))

In [ ]:
adf.createOrReplaceTempView('adf')
kdf.createOrReplaceTempView('kdf')

In [ ]:
kdf.show()

In [ ]:
sql_query = "SELECT l.*, r.killed_by from adf l JOIN kdf r ON l.`match_id` = r.`match_id` and l.player_name=r.victim_name  "
join_df = spark.sql(sql_query)

In [ ]:
join_df.show()

In [ ]:
join_df.count()

In [ ]:
import pandas as pd

padf = pd.read_csv(agg_match_stats)
pkdf = pd.read_csv(kill_match_stats)

In [ ]:
pjoin_df = pd.merge(padf, pkdf, how='inner', left_on=['match_id', 'player_name'], right_on=['match_id', 'victim_name'])

In [ ]:
len(pjoin_df)

In [ ]:
# val mySchema = StructType(Array(
#  StructField("id", IntegerType),
#  StructField("name", StringType),
#  StructField("year", IntegerType),
#  StructField("rating", DoubleType),
#  StructField("duration", IntegerType)
# ))
movie_schema = StructType().add("id", "integer")\
.add("name", "string").add("year", "integer").add("rating", "double").add("duration", "integer")



df = spark.read.schema(movie_schema).csv('/Users/phaneendra/Downloads/moviedata.csv', inferSchema=True, header=True)

In [ ]:
from pyspark.sql import functions as F

newDf2 = df.withColumn("value", F.to_json(F.struct([F.when(F.col(x)!="  ",df[x]).otherwise(None).alias(x) for x in df.columns])))
newDf2.show(truncate=False)

In [ ]:
newDf2.columns

In [ ]:
#Worked
newDf2.selectExpr("CAST(year AS INTEGER)", "CAST(value AS STRING)") \
  .write \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092") \
  .option("topic", "test_kafka_spark") \
  .save()



In [ ]:
#Worked
newDf2.selectExpr("CAST(year AS STRING) AS key", "to_json(struct(*)) AS value") \
  .write \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092") \
  .option("topic", "test_kafka_spark_2") \
  .save()

In [ ]:
#Try for SalesRecords

sdf = spark.read.csv('/Users/phaneendra/Downloads/100000SalesRecords.csv', inferSchema=True, header=True)
sdf.columns

In [ ]:
sdf.selectExpr("CAST(Region AS STRING) AS key", "to_json(struct(*)) AS value") \
  .write \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092") \
  .option("topic", "test_kafka_spark_3") \
  .save()

In [ ]:
dft1 = spark.readStream.format("kafka").option("kafka.bootstrap.servers", "localhost:9092").option("subscribe", "topic-one").option("startingOffsets", "latest").load()



In [ ]:
dft1.writeStream.format("kafka").option("topic", "topic-one").option("kafka.bootstrap.servers", "localhost:9092").option("checkpointLocation","/tmp/").start()




In [ ]:
prob_schema = StructType().add("CUST_ID", "string").add("OUTBUFFER__1", "string")
# Read CSV files from set path
dfCSV = spark.readStream.option("sep", ",").option("header", "true").schema(prob_schema).csv("/Users/phaneendra/Downloads/prob*.csv")


In [ ]:
dfCSV.createOrReplaceTempView("prob")
total = spark.sql("select count(CUST_ID) from prob")


In [ ]:
dfCSV \
.write  \
.format("kafka") \
.option("kafka.batch.size", 5000) \
.option("kafka.bootstrap.servers", "localhost:9092") \
.option("kafka.request.timeout.ms", 120000) \
.option("topic", "raw") \
.option("checkpointLocation", "/mnt/telemetry/cp.txt") \
.save()

In [ ]:
query = total.writeStream.outputMode("complete").format("console").start()
query.awaitTermination()

In [ ]:
query.close()

In [ ]:
dfCSV.selectExpr("CAST(CUST_ID AS STRING) AS key", "to_json(struct(*)) AS value").
  writeStream
  .format("kafka")
  .option("topic", "topic-one")
  .option("kafka.bootstrap.servers", "localhost:9092")
  .start()

In [ ]:
#Hive

val ss = SparkSession
.builder()
.appName(" Hive example")
.config("hive.metastore.uris", "thrift://localhost:9083")
.enableHiveSupport()
.getOrCreate()
df = ss.read.table("db_name.table_name")




In [ ]:

from impala.dbapi import connect
hive_connection = connect(host="quickstart.cloudera", port=10000, user="phani", database="bdp", auth_mechanism="GSSAPI", use_kerberos=True, kerberos_service_name="hive")



In [ ]:

from pyhive import hive
conn = hive.Connection(host="quickstart.cloudera", port=10000, username="phani", database="bdp", auth="KERBEROS", kerberos_service_name="hive")
cur = conn.cursor()

res = cur.execute("CREATE TABLE sales (Region STRING,Country STRING) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LINES TERMINATED BY '\n' STORED as textfile")

cur.execute("LOAD DATA INPATH '/user/phani/100000SalesRecords1.csv' INTO TABLE sales;")

In [ ]:
#HIve

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .enableHiveSupport() \
    .getOrCreate()

df.write.mode('Overwrite').saveAsTable('spotify_track_details')
spark.sql('select * from spotify_track_details limit 2').show()

